In [1]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

CPU times: user 1.66 s, sys: 334 ms, total: 1.99 s
Wall time: 2.71 s


In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords

In [15]:
stops = set(stopwords.words("english"))
def cleantext(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    tx = text.replace('&amp',' ')
    text = re.sub(r"[^0-9a-zA-Z]+",' ',text)
    text = text.split()
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [20]:
%%time
path = "/content/drive/My Drive/SharedTasksParticipation/SharedTask@Constraint@AAAI2021/"
train_filename= path + "Constraint_English_Train.xlsx"
val_filename = path + "Constraint_English_Val .xlsx"
test_filename = path + "Constraint_English_Test.xlsx"

train = pd.read_excel(train_filename)
val = pd.read_excel(val_filename)
test = pd.read_excel(test_filename)

CPU times: user 409 ms, sys: 657 µs, total: 410 ms
Wall time: 3.69 s


In [21]:
train['train_label']= train.label.replace({'real':0,'fake':1})
val['val_label']= val.label.replace({'real':0,'fake':1})

In [22]:
train['cleaned'] = train['tweet'].map(lambda x: cleantext(x))
val['cleaned'] = val['tweet'].map(lambda x: cleantext(x))
test['cleaned'] = test['tweet'].map(lambda x: cleantext(x))

In [23]:
total_data=pd.concat([train.cleaned, val.cleaned, test.cleaned], axis=0)

In [24]:
train

,id,tweet,label,train_label,cleaned
0,1,The CDC currently reports 99031 deaths. In gen...,real,0,cdc currently reports 99031 deaths general dis...
1,2,States reported 1121 deaths a small rise from ...,real,0,states reported 1121 deaths small rise last tu...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,1,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,0,indiafightscorona 1524 covid testing laborator...
4,5,Populous states can generate large case counts...,real,0,populous states generate large case counts loo...
...,...,...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake,1,tiger tested positive covid 19 please stay awa...
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake,1,autopsies prove covid 19 blood clot pneumonia ...
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake,1,post claims covid 19 vaccine already developed...
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake,1,aamir khan donate 250 cr pm relief cares fund


In [25]:
val

,id,tweet,label,val_label,cleaned
0,1,Chinese converting to Islam after realising th...,fake,1,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,1,11 13 people diamond princess cruise ship inti...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,1,covid 19 caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,1,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,0,6 10 sky edconwaysky explains latest covid19 d...
...,...,...,...,...,...
2135,2136,Donald Trump wrongly claimed that New Zealand ...,fake,1,donald trump wrongly claimed new zealand big s...
2136,2137,Current understanding is #COVID19 spreads most...,real,0,current understanding covid19 spreads mostly p...
2137,2138,Nothing screams “I am sat around doing fuck al...,fake,1,nothing screams sat around fuck lockdown quite...
2138,2139,Birx says COVID-19 outbreak not under control ...,fake,1,birx says covid 19 outbreak control people move


In [26]:
test

,id,tweet,label,cleaned
0,1,Our daily update is published. States reported...,real,daily update published states reported 734k te...
1,2,Alfalfa is the only cure for COVID-19.,fake,alfalfa cure covid 19
2,3,President Trump Asked What He Would Do If He W...,fake,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,real,states reported 630 deaths still seeing solid ...
4,5,This is the sixth time a global health emergen...,real,sixth time global health emergency declared in...
...,...,...,...,...
2135,2136,#CoronaVirusUpdates: State-wise details of Tot...,real,coronavirusupdates state wise details total co...
2136,2137,Tonight 12(midnight) onwards Disaster Manageme...,fake,tonight 12 midnight onwards disaster managemen...
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 E...,real,296 new cases covid19nigeria plateau 85 enugu ...
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...,real,rt cdcemergency dyk cdcgov one stop shop covid...


In [27]:
total_data

0       cdc currently reports 99031 deaths general dis...
1       states reported 1121 deaths small rise last tu...
2       politically correct woman almost uses pandemic...
3       indiafightscorona 1524 covid testing laborator...
4       populous states generate large case counts loo...
                              ...                        
2135    coronavirusupdates state wise details total co...
2136    tonight 12 midnight onwards disaster managemen...
2137    296 new cases covid19nigeria plateau 85 enugu ...
2138    rt cdcemergency dyk cdcgov one stop shop covid...
2139    half pregnant women recently admitted uk hospi...
Name: cleaned, Length: 10700, dtype: object

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer

def text_encoding(texts, labels, padding_length):
  tokenizer = Tokenizer()
  
  # Tokenizer is fitted into the texts
  tokenizer.fit_on_texts(texts)

  # Encoding texts into integer sequences
  sequences = tokenizer.texts_to_sequences(texts)
  # Pad sequences
  corpus = keras.preprocessing.sequence.pad_sequences(sequences, value=0.0,
                                                      padding='post', maxlen= padding_length)

  return corpus, labels, tokenizer

##
#max_words = 20000 # Based on number of unique words in the dataset
padding_length = 100
corpus, labels, tokenizer = text_encoding(train.cleaned, train.train_label, padding_length)

In [32]:
train_data, train_labels = corpus, labels

In [35]:
print(train_data, train_labels)

[[  69  215  203 ...    0    0    0]
 [  12   18 6790 ...    0    0    0]
 [4792 1770  373 ...    0    0    0]
 ...
 [ 216  104    1 ...    0    0    0]
 [2806  899 2685 ...    0    0    0]
 [1560   42   83 ...    0    0    0]] 0       0
1       0
2       1
3       0
4       0
       ..
6415    1
6416    1
6417    1
6418    1
6419    0
Name: train_label, Length: 6420, dtype: int64


In [36]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 14149


In [37]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-02-15 17:13:44--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-15 17:13:44--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-15 17:13:45--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [38]:
GLOVE_EMB = '/content/glove.6B.100d.txt'

embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
  values = line.split()
  word = value = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [39]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [40]:
## Defining Callbacks
def define_callback():
  keras.backend.clear_session()
  accuracy_threshold = .9999

  class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>accuracy_threshold):
          print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
          self.model.stop_training = True

  acc_callback = myCallback()

  return acc_callback

# Define Checkpoint
def define_checkpoint():
  filepath = "ModelOnOurData.h5"
  checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, save_weights_only=False, mode='max')
  return checkpoint

In [41]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.optimizers import Adam

In [42]:
# Create Model
def create_model(embedding_dim):
  keras.backend.clear_session()

  model = keras.models.Sequential([
      keras.layers.Embedding(vocab_size, embedding_dim, input_length = padding_length),
      #keras.layers.Conv1D(64, 5, activation='relu'),
      #keras.layers.MaxPooling1D(5),
      #keras.layers.Bidirectional(LSTM(32, dropout=0.4)),
      #keras.layers.Dense(64, activation='relu'),
      #keras.layers.Dropout(0.5),
      keras.layers.Dense(64, activation='relu'),
      keras.layers.Flatten(),
      keras.layers.Dense(2, activation = 'softmax')
  ])

  model.summary()
  return model


# Compile Model
def compile_model(model):
  model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
  return model


In [43]:
%%time
## Callbacks are really important 

def run_callback_list():
  acc_callback = define_callback()
  checkpoint = define_checkpoint()
  return [acc_callback, checkpoint] 

callback_list = run_callback_list()

def run_model():
    embedding_dim = 100
    model = create_model(embedding_dim)
    model = compile_model(model)
    return model

## Create, Compile and Visualize the model
cnn_lstm_model = run_model() ## 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1414900   
_________________________________________________________________
dense (Dense)                (None, 100, 64)           6464      
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 12802     
Total params: 1,434,166
Trainable params: 1,434,166
Non-trainable params: 0
_________________________________________________________________
CPU times: user 450 ms, sys: 334 ms, total: 784 ms
Wall time: 5.6 s


In [44]:
cnn_lstm_model.layers[0].set_weights([embedding_matrix])
cnn_lstm_model.layers[0].trainable = False

In [45]:
cnn_lstm_model_history = cnn_lstm_model.fit(train_data,
                              train_labels,
                              epochs = 50,
                              batch_size = 64,
                              verbose = 1,
                              validation_split = 0.15,
                              callbacks = callback_list)

Epoch 1/50
86/86 [==============================] - 3s 16ms/step - loss: 0.6305 - accuracy: 0.6410 - val_loss: 0.5497 - val_accuracy: 0.7456

Epoch 00001: val_accuracy improved from -inf to 0.74559, saving model to ModelOnOurData.h5
Epoch 2/50
86/86 [==============================] - 1s 14ms/step - loss: 0.5337 - accuracy: 0.7508 - val_loss: 0.5012 - val_accuracy: 0.7632

Epoch 00002: val_accuracy improved from 0.74559 to 0.76324, saving model to ModelOnOurData.h5
Epoch 3/50
86/86 [==============================] - 1s 14ms/step - loss: 0.4785 - accuracy: 0.7734 - val_loss: 0.4595 - val_accuracy: 0.7913

Epoch 00003: val_accuracy improved from 0.76324 to 0.79128, saving model to ModelOnOurData.h5
Epoch 4/50
86/86 [==============================] - 1s 14ms/step - loss: 0.4247 - accuracy: 0.8131 - val_loss: 0.4235 - val_accuracy: 0.8120

Epoch 00004: val_accuracy improved from 0.79128 to 0.81205, saving model to ModelOnOurData.h5
Epoch 5/50
86/86 [==============================] - 1s 14ms

In [46]:
val_sequences = tokenizer.texts_to_sequences(val.cleaned)
print(val_sequences[0:10])
val_data = keras.preprocessing.sequence.pad_sequences(val_sequences, value=0.0,
                                                      padding='post', maxlen= padding_length)
val_labels = val['val_label']

print(val_data[:5], val_data.shape)
print(val_labels[:5], val_labels.shape)

[[234, 9592, 4338, 828, 424, 4, 73], [106, 239, 8, 5084, 3991, 1683, 1347, 41, 236, 9, 1116, 1126, 16, 40, 346, 12], [1, 2, 392, 2526, 29, 859, 5044], [2014, 2032, 7757, 4078, 10896, 178, 43, 1, 2, 3937, 2174, 1010, 2457, 68, 23, 3636], [55, 63, 968, 1053, 110, 5, 38, 67, 2639, 77, 4, 38], [24, 1515, 135, 66, 959, 374, 851, 236, 44, 1971, 44, 1799, 637, 311, 42, 298, 3796, 3877, 2468, 4320, 5694], [27, 7, 24, 449, 5, 575, 1375, 290, 13, 31, 445, 57, 502, 6074, 75, 66, 17, 339, 371, 204, 122, 12, 375, 1700, 416, 156, 132], [39, 5, 297, 10464, 4462, 2419, 260, 166, 111, 8, 3791], [70, 262, 6100, 1111, 5572, 257, 931, 2911, 18, 1588, 2084, 315, 107, 3914, 2809, 248, 75], [2497, 448, 2176, 2420, 35, 284, 301]]
[[  234  9592  4338   828   424     4    73     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0 

In [49]:
cnn_lstm_model=load_model("/content/ModelOnOurData.h5")
val_ls, val_acc = cnn_lstm_model.evaluate(val_data, val_labels)
#print(val_labels)
#print(val_ls, val_acc)
pred = np.argmax(cnn_lstm_model.predict(val_data), axis=-1)
#print(pred)
print(confusion_matrix(pred, val_labels))
#plot_confusion_matrix(confusion_matrix(val['label'],pred),target_names=['real','fake'], normalize = False, \
                      #title = 'Confusion matix of SVM on val data')


#svm_val_misclass_df = val_ori[pred!=val['label']]

67/67 [==============================] - 0s 3ms/step - loss: 0.2187 - accuracy: 0.9220
[[1040   87]
 [  80  933]]


# Prediction on test set

In [50]:
test_sequences = tokenizer.texts_to_sequences(test.cleaned)
print(test_sequences[0:10])
test_data = keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0,
                                                      padding='post', maxlen= padding_length)

[[48, 34, 97, 12, 18, 9, 8531, 6, 3, 9714, 10, 251, 223, 1386, 2716, 80, 86, 83, 298, 406], [134, 1, 2], [68, 43, 633, 164, 2397, 4, 309, 4], [12, 18, 10, 113, 1415, 1972, 156, 984, 79, 105, 1879, 3011, 311, 42, 1649, 1530, 316, 69, 1498, 1692, 1879, 127, 86, 30, 79, 105, 1191], [3631, 86, 328, 20, 437, 1532, 535, 20, 2547, 1918, 354, 141], [292, 7603, 3401, 1562, 32, 1, 2], [704, 1785, 493, 3711, 84, 1948, 251, 3711, 84, 12, 25, 251, 12, 25, 493, 1540, 10360, 23, 493, 5099, 251], [67, 1692, 351, 6, 156, 52, 693, 3735, 1706, 35, 2200, 539, 1208], [48, 34, 97, 310, 13, 81, 61, 9, 62, 18, 183, 83, 139, 11, 214, 180, 9, 23, 203, 12, 25, 9, 53], [576, 19, 37, 536, 2824, 2100, 1287, 17, 20, 295, 228, 2604, 945, 1287, 4115, 17, 13, 9167, 1405, 687, 236, 9, 855, 251, 4115, 13085, 926, 17, 2491, 261, 236]]


In [51]:
pred= np.argmax(cnn_lstm_model.predict(test_data), axis=-1)

In [52]:
jongu = pd.DataFrame(pred, columns=['ll'])

In [ ]:
jongu

,ll
0,0
1,1
2,1
3,0
4,0
...,...
2135,0
2136,0
2137,0
2138,0


In [ ]:
test['label']= jongu.ll.replace({0:'real',1:'fake'})

In [ ]:
test.head(15)

,id,tweet,cleaned,label
0,1,Our daily update is published. States reported...,daily update published states reported 734k te...,real
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid 19,fake
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...,fake
3,4,States reported 630 deaths. We are still seein...,states reported 630 deaths still seeing solid ...,real
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...,real
5,6,Low #vitaminD was an independent predictor of ...,low vitamind independent predictor worse progn...,real
6,7,A common question: why are the cumulative outc...,common question cumulative outcome numbers sma...,real
7,8,The government should consider bringing in any...,government consider bringing new national lock...,real
8,9,Our daily update is published. We’ve now track...,daily update published tracked 2 9 million tes...,real
9,10,Breakdown of testing: 4 air crew 97 hotel &amp...,breakdown testing 4 air crew 97 hotel amp heal...,real


In [ ]:
test[['id','label']]

In [ ]:
test['label'].value_counts()

real    1109
fake    1031
Name: label, dtype: int64

In [ ]:
with open('svm(94.62).txt', 'w') as f:
  f.write("id,label\n")
  for i in range(len(pred_test)):
    id = test['id'][i]
    v = pred_test[i]
    f.write("%d,%s\n"%(id, v))